# TASK:
## given a satellite image (x-band SAR):
## A) Show the image.
## B) Denoise the image aplying spatial filters.
## C) Apply false color for enhanced contrast.


In [ ]:
'''imports'''
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib import cm
import cmapy

image_path = 'ICEYE_GRD_SLH_30180_20200603T035624a.tif'

## Defining functions.


In [ ]:
def read_image(path):
    '''read the image from path and prints information relative to the image.'''
    image = cv2.imread(path, -1)
    print("ORIGINAL IMAGE \n -Shape: ", image.shape, "\n -Data type: ", image.dtype, "\n -Value range: ", (image.min(), image.max()))
    return image

def heat_equation_convolution(image, t, c):
    '''apply the 2D convolution given by the heat equation (explained below):
        PARAMETERS:
            -t: time (discretized, number of steps to perform)
            -c: Spreading weight (heat spread "speed")'''
    
    Laplacian = np.array([[0, -1, 0],
                        [-1, 4, -1],
                         [0, -1, 0]])
    A = image.copy()
    burnt = np.zeros_like(image)
    for i in range(1,t):
        burnt = A - c*cv2.filter2D(A,-1, Laplacian)
        A = burnt
    return A.astype(np.uint16)



## Interpreting the data

In [ ]:
image = read_image(image_path)

In [ ]:
'''showing the original image... NOISY!'''
img = image*255/65535 #normalizing to 0-1
plt.figure(figsize = (20,20))
plt.imshow(img, cmap = 'gray')

we see as expected that the image is quite noisy, mostly salt-an-pepper noise.

## Typical filters: median and gaussian blur

In [ ]:
'''median blur (baseline)'''
median_blur = cv2.medianBlur(image, 5) #normalized
print(median_blur.max())

In [ ]:
'''we can see that median blur greatly reduces "salt and pepper" noise'''

f = plt.figure(figsize = (50,40))
f.add_subplot(1,2,1)
plt.imshow(image*255/image.max(), cmap = 'gray')
f.add_subplot(1,2,2)
plt.imshow(median_blur*255/median_blur.max(), cmap = 'gray')
plt.show(block=True)

Original image vs median filter

In [ ]:
'''now the same with gaussian blur'''
gaussian_blur = cv2.GaussianBlur(image, ksize=(15,15),sigmaX=0)

In [ ]:
'''we see that the nosie removal is similar, let's compare side by side'''
f = plt.figure(figsize = (50,40))
f.add_subplot(1,2,1)
plt.imshow(image*255/image.max(), cmap = 'gray')
f.add_subplot(1,2,2)
plt.imshow(gaussian_blur*255/gaussian_blur.max(), cmap = 'gray')
plt.show(block=True)

original image vs gaussian blur

In [ ]:
f = plt.figure(figsize = (40,20))
f.add_subplot(1,2,1)
plt.imshow(median_blur*255/median_blur.max(), cmap = 'gray')
f.add_subplot(1,2,2)
plt.imshow(gaussian_blur*255/gaussian_blur.max(), cmap = 'gray')
plt.show(block=True)
'''gaussian blur seems to keep more information'''

### median filter / gaussian blur
we see that gaussian blur does better than median blur, since we can apply a bigger kernel without blurring the edges of the image.

## Heat equation:
intuition: pixel value -> temperature (energy). Aplying the heat equation below we can "spread" the pixel value among its neighbours. 
$$\frac{\delta u}{\delta t} = c \nabla^2 u$$
in the discrete 2D case, after expansion at a discrete timestep k:
$$u^{k+1}_{i,j} = u^{k}_{i,j} + c\left( u^{k}_{i+1,j} + u^{k}_{i-1,j} + u^{k}_{i,j+1} + u^{k}_{i,j-1}-4u^{k}_{i,j}\right)$$

effectively this makes an analogy between pixel values and temperature, the higher the value, the higher the temperature, and uses the heat equation to interact with its neighbours,
which is exactly the function I defined above, let's see how it does:
(convergence condition: $c < \frac{1}{4}$)

Because I only have 8gb ram this keeps crashing every 2 plots or so, I'm copy-pasting functions and imports below for convenience.

In [ ]:
'''imports'''
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib import cm
import cmapy #library to use matplotlib colormaps in opencv

image_path = 'ICEYE_GRD_SLH_30180_20200603T035624a.tif'

def read_image(path):
    image = cv2.imread(path, -1)
    print("ORIGINAL IMAGE \n -Shape: ", image.shape, "\n -Data type: ", image.dtype, "\n -Value range: ", (image.min(), image.max()))
    return image

def heat_equation_convolution(image, t, c):
    Laplacian = np.array([[0, -1, 0],
                        [-1, 4, -1],
                         [0, -1, 0]])
    A = image.copy()
    burnt = np.zeros_like(image)
    for i in range(t):
        burnt = A - c*cv2.filter2D(A,-1, Laplacian)
        A = burnt
    return A.astype(np.uint16)

image = read_image(image_path)

In [ ]:
#burnt = heat_equation_convolution(image,100, .15)
burnt_halftime = heat_equation_convolution(image, 50, .15)
burnt = heat_equation_convolution(image, 50, .09)
print(burnt_halftime.max())
#burnt2 = burnt.astype(np.uint16)

In [ ]:
f = plt.figure(figsize = (40,20))
f.add_subplot(1,2,1)
plt.imshow(image*255/image.max(), cmap = 'gray')
f.add_subplot(1,2,2)
plt.imshow(burnt*255/burnt.max(), cmap = 'gray')
plt.show(block=True)


### original image / heat equation denoising
we see that denoising is very good in this case, conserving edges quite well.

In [ ]:
'''now comparing with gaussian_blur'''
f = plt.figure(figsize = (40,20))
f.add_subplot(1,2,1)
plt.imshow(burnt_halftime*255/burnt_halftime.max(), cmap = 'gray')
f.add_subplot(1,2,2)
plt.imshow(gaussian_blur*255/gaussian_blur.max(), cmap = 'gray')
plt.show(block=True)


### heat equation denoising / gaussian blur
heat equation seems to do better.

In [ ]:
'''comparing denoised image vs original image -> wow! it's hard to take out more noise without blurring the image too much, but it's a quite good result'''
burnt_color = cv2.applyColorMap((burnt_halftime).astype(np.uint8), cmapy.cmap('CMRmap', rgb_order = True))
image_color = cv2.applyColorMap(image.astype(np.uint8), cmapy.cmap('CMRmap', rgb_order = True))
f = plt.figure(figsize = (40,20))
f.add_subplot(1,2,1)
plt.imshow(burnt_color)
f.add_subplot(1,2,2)
plt.imshow(gaussian_color)
plt.show(block=True)

here we see the power of this denoising technique, comparison between the original image and the heat-equation method above (in colormap).

In [ ]:
cv2.imwrite('HEAT+Color.tif', im_color)

In [ ]:
f = plt.figure(figsize = (40,20))
f.add_subplot(1,2,1)
plt.imshow(burnt_halftime*255/burnt_halftime.max(), cmap = 'gray')
f.add_subplot(1,2,2)
plt.imshow(burnt*255/burnt.max(), cmap = 'gray')
plt.show(block=True)

## Bilateral filters
copy-pasting again for convenience:

In [ ]:
'''imports'''
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib import cm
import cmapy #library to use matplotlib colormaps in opencv

image_path = 'ICEYE_GRD_SLH_30180_20200603T035624a.tif'

def read_image(path):
    image = cv2.imread(path, -1)
    print("ORIGINAL IMAGE \n -Shape: ", image.shape, "\n -Data type: ", image.dtype, "\n -Value range: ", (image.min(), image.max()))
    return image

def heat_equation_convolution(image, t, c):
    Laplacian = np.array([[0, -1, 0],
                        [-1, 4, -1],
                         [0, -1, 0]])
    A = image.copy()
    burnt = np.zeros_like(image)
    for i in range(t):
        burnt = A - c*cv2.filter2D(A,-1, Laplacian)
        A = burnt
    return A.astype(np.uint16)

image = read_image(image_path)

In [ ]:
bilateral = cv2.bilateralFilter(image.astype(np.uint8),17, 300, 1000)

heat = heat_equation_convolution(image, 50, .15)


heat_color = cv2.applyColorMap((heat).astype(np.uint8), cmapy.cmap('CMRmap', rgb_order = True))
bilateral_color = cv2.applyColorMap(bilateral.astype(np.uint8), cmapy.cmap('CMRmap', rgb_order = True))

f=plt.figure(figsize = (40,20))
f.add_subplot(1,2,1)
plt.imshow(heat_color)
f.add_subplot(1,2,2)
plt.imshow(bilateral_color)
plt.show()

Bilateral filter is better at conserving information and smoothes very bright pixels better

In [ ]:
bilateral = cv2.bilateralFilter(image.astype(np.uint8),17, 300, 1000)
bilateral_color = cv2.applyColorMap(bilateral.astype(np.uint8), cmapy.cmap('CMRmap', rgb_order = False))
cv2.imwrite('bilateral_x1_17_color.tif', bilateral_color)

# CONCLUSIONS:
### -Median and gaussian blur both do decently, but seem to be outperformed by the heat equation method and the bilateral filter.
### -Bilateral filter and heat equation convolution have similar noise reduction capabilities (being bilateral filter somewhat better, it outperforms other filters by applying spatial filter and a color (or frequency) filter, which helps it to prevent blurring the edges).
### -I found the heat equation method most interesting due to its physical interpretation and ease of visualization (kind of like the brightest pixel values melt to its neighbours), but I'm going to use the bilateral filter for the task due to its bigger signal to noise ratio.

I will be posting the main method and upload this too for consulting.

### What I really would have liked to do is to train an autoencoder on images like this to train in an unsupervised manner to perform denoising task. But it's too computationally expensive and you need a big dataset for it to learn properly.
